Given a string s, partition s such that every substring of the partition is a palindrome. Return all possible palindrome partitioning of s.

 

Example 1:

Input: s = "aab"
Output: [["a","a","b"],["aa","b"]]
Example 2:

Input: s = "a"
Output: [["a"]]
 

Constraints:

1 <= s.length <= 16
s contains only lowercase English letters.

In [ ]:
# recursive solution.
class Solution:
    def __init__(self):
        self.res = []

    def partition(self, s: str) -> list[list[str]]:
        self.partition_rec(s, 0, [], len(s))
        return self.res

    def is_palindrome(self, sub: str) -> bool:
            return sub == sub[::-1]
    
    def partition_rec(self, s, start_ind, cur_arr, n):
        if start_ind == n:
            self.res.append(cur_arr.copy())
            return
        
        # consider all values method.
        # you are gonna take values upto 0 th index, 1st index, ... upto ith index.
        # if the current partition is palidrome.
        # add them to the curr_arr. and call the recurssion.
        for end in range(start_ind, n):
            if self.is_palindrome(s[start_ind: end + 1]):
                cur_arr.append(s[start_ind: end+1])
                self.partition_rec(s, end + 1, cur_arr, n)
                cur_arr.pop()

# tc:
# - recurssion: O(2 ^ n)
# - palindrom check - O(n)
# - arr copy - O(n)
# tc - O(n * 2^n)[palindrom check] + O(n * 2 ^ n)[arr copy] = O(n * 2 ^ n)

# sc:
# - axilory - O(n)
# - output - O(2 ^ n) # number of substrings 
# - cur_arr - can take max O(n)
# sc - O(n * 2^n)

In [ ]:
# we can use tabulation to reduce the palindrom check time.
# but the time complexity will still be O(n * 2^n) as we are

# is_pal[i][j] = True if s[i:j+1] is a palindrome.
class Solution:
    def partition(self, s: str) -> list[list[str]]:
        n = len(s)
        
        # Step 1: Precompute palindrome table
        is_pal = [[False] * n for _ in range(n)]
        
        # * We want to fill a 2D table `is_pal[i][j]` that is True if substring `s[i..j]` is a palindrome.
        # * We check from the **end of the string backward for start index i**, and forward for end index j ≥ i.
        # * Condition to mark `is_pal[i][j] = True`:

        # 1. **Characters at the edges are the same:** `s[i] == s[j]`

        # 2. And either:

        #     * The substring length is less than 3 (`j - i < 3`), which means:

        #     * length 1 substring: always palindrome, e.g., "a"
        #     * length 2 substring: palindrome if chars are equal, e.g., "aa" - NOTE: this is checked by the firs condifiotn s[start] == s[end].
        #     * OR the inside substring `s[i+1..j-1]` is already palindrome: `is_pal[i+1][j-1] == True`

        for start in range(n-1, -1, -1):
            for end in range(start, n):
                if s[start] == s[end] and (end - start < 3 or is_pal[start+1][end-1]):
                    is_pal[start][end] = True
        
        res = []
        path = []

        # Step 2: Backtracking using precomputed palindrome table
        def backtrack(start):
            if start == n:
                res.append(path[:])
                return
            
            for end in range(start, n):
                if is_pal[start][end]:
                    path.append(s[start:end+1])
                    backtrack(end+1)
                    path.pop()
        
        backtrack(0)
        return res
    
# tc:
# - precompute palindrom table: O(n^2)
# - backtracking: O(2^n)
# overall: O(n^2 + 2^n) ~= O(2^n) as n is small.

# sc:
# - palindrom table: O(n^2)
# - output: O(2^n)

Got it! Let me explain **this DP palindrome table construction** and then do a dry run on an example string.

---

## Explanation of this snippet:

```python
for i in range(n-1, -1, -1):          # iterate backwards on start index i
    for j in range(i, n):             # iterate forwards on end index j >= i
        if s[i] == s[j] and (j - i < 3 or is_pal[i+1][j-1]):
            is_pal[i][j] = True
```

---

### What it does?

* We want to fill a 2D table `is_pal[i][j]` that is True if substring `s[i..j]` is a palindrome.
* We check from the **end of the string backward for start index i**, and forward for end index j ≥ i.
* Condition to mark `is_pal[i][j] = True`:

  1. **Characters at the edges are the same:** `s[i] == s[j]`

  2. And either:

     * The substring length is less than 3 (`j - i < 3`), which means:

       * length 1 substring: always palindrome, e.g., "a"
       * length 2 substring: palindrome if chars are equal, e.g., "aa"
     * OR the inside substring `s[i+1..j-1]` is already palindrome: `is_pal[i+1][j-1] == True`

---

### Why `j - i < 3`?

* If substring length is 1 or 2, just need to check edge chars (`s[i] == s[j]`) to confirm palindrome.
* For length > 2, the substring is palindrome only if the inner substring is palindrome.

---

## Dry Run: Example `s = "abba"`

Length `n = 4`

Initialize `is_pal` as 4x4 False matrix.

---

### Iteration:

* `i = 3` (start from last index)

  * `j = 3`

    * `s[3] == s[3]` → 'a' == 'a' (True)
    * `j - i < 3` → 0 < 3 (True)
    * So `is_pal[3][3] = True` (substring "a")

* `i = 2`

  * `j = 2`

    * `s[2] == s[2]` → 'b' == 'b' (True)
    * `j - i < 3` → 0 < 3 (True)
    * `is_pal[2][2] = True` ("b")
  * `j = 3`

    * `s[2] == s[3]` → 'b' == 'a' (False)
    * `is_pal[2][3]` stays False ("ba")

* `i = 1`

  * `j = 1`

    * `s[1] == s[1]` → 'b' == 'b' (True)
    * `j - i < 3` → 0 < 3 (True)
    * `is_pal[1][1] = True` ("b")
  * `j = 2`

    * `s[1] == s[2]` → 'b' == 'b' (True)
    * `j - i < 3` → 1 < 3 (True)
    * `is_pal[1][2] = True` ("bb")
  * `j = 3`

    * `s[1] == s[3]` → 'b' == 'a' (False)
    * `is_pal[1][3] = False` ("bba")

* `i = 0`

  * `j = 0`

    * `s[0] == s[0]` → 'a' == 'a' (True)
    * `j - i < 3` → 0 < 3 (True)
    * `is_pal[0][0] = True` ("a")
  * `j = 1`

    * `s[0] == s[1]` → 'a' == 'b' (False)
    * `is_pal[0][1] = False` ("ab")
  * `j = 2`

    * `s[0] == s[2]` → 'a' == 'b' (False)
    * `is_pal[0][2] = False` ("abb")
  * `j = 3`

    * `s[0] == s[3]` → 'a' == 'a' (True)
    * `j - i < 3` → 3 < 3 (False)
    * Check inner substring `is_pal[1][2]` = True (from above)
    * So `is_pal[0][3] = True` ("abba")

---

### Final `is_pal` matrix:

| i\j | 0    | 1     | 2     | 3     |
| --- | ---- | ----- | ----- | ----- |
| 0   | True | False | False | True  |
| 1   |      | True  | True  | False |
| 2   |      |       | True  | False |
| 3   |      |       |       | True  |

---

### Summary:

* This DP lets you check any substring palindrome in **O(1)** after $O(n^2)$ preprocessing.
* Important for palindrome partitioning problems to speed up palindrome checks inside recursive/backtracking or DP solutions.

---

If you want, I can do the dry run on another example or explain how this integrates with backtracking!
